In [2]:
import sklearn
import seaborn
import pandas

iris = pandas.read_csv('data/iris.csv')

In [6]:
iris['SepalArea']=iris['SepalLength']*iris['SepalWidth']
iris['PetalArea']=iris['PetalLength']*iris['PetalWidth']

In [7]:
iris


,species,SepalLength,SepalWidth,PetalLength,PetalWidth,SepalArea,PetalArea
0,setosa,5.1,3.5,1.4,0.2,17.85,0.28
1,setosa,4.9,3.0,1.4,0.2,14.70,0.28
2,setosa,4.7,3.2,1.3,0.2,15.04,0.26
3,setosa,4.6,3.1,1.5,0.2,14.26,0.30
4,setosa,5.0,3.6,1.4,0.2,18.00,0.28
...,...,...,...,...,...,...,...
145,virginica,6.7,3.0,5.2,2.3,20.10,11.96
146,virginica,6.3,2.5,5.0,1.9,15.75,9.50
147,virginica,6.5,3.0,5.2,2.0,19.50,10.40
148,virginica,6.2,3.4,5.4,2.3,21.08,12.42
